In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('\tmp\data', one_hot=True)

Extracting 	mp\data/train-images-idx3-ubyte.gz
Extracting 	mp\data/train-labels-idx1-ubyte.gz
Extracting 	mp\data/t10k-images-idx3-ubyte.gz
Extracting 	mp\data/t10k-labels-idx1-ubyte.gz


In [18]:
tf.reset_default_graph()
learning_rate = 0.01
num_steps = 1000
batch_size = 128
step_display = 100

num_input = 28*28
num_classes = 10
dropout = 0.75

sess = tf.Session()

# Create a dataset tensor from the images and the labels
dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images, mnist.train.labels))

# Automatically refill the data queue when empty
dataset = dataset.repeat()

# Create batches of data
dataset = dataset.batch(batch_size)

# Prefetch data for faster consumption
dataset = dataset.prefetch(batch_size)

# Create an iterator over the dataset
iterator = dataset.make_initializable_iterator()

# Initialize the iterator
sess.run(iterator.initializer)

X,Y = iterator.get_next()

In [19]:
def conv_net(x, num_classes, droupout, reuse, is_training):
    
    with tf.variable_scope('Conv_net', reuse = reuse):
        
        x = tf.reshape(x, [-1,28,28,1])
        
        layer1 = tf.layers.conv2d(x, 32, 5, activation = tf.nn.relu)
        layer1 = tf.layers.max_pooling2d(layer1,2,2)
        
        layer2 = tf.layers.conv2d(layer1, 64, 3, activation = tf.nn.relu)
        layer2 = tf.layers.max_pooling2d(layer2,2,2)
        
        fc = tf.contrib.layers.flatten(layer2)
        
        fc = tf.layers.dense(fc, 1024)
        fc = tf.layers.dropout(fc, rate=dropout, training=is_training)
        
        out = tf.layers.dense(fc, num_classes)
        out = tf.nn.softmax(out) if not is_training else out
        
        return out
    
logits_train = conv_net(X, num_classes, dropout, reuse=False, is_training=True)
logits_test = conv_net(X, num_classes, dropout, reuse=True, is_training=False)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logits_train, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
init = tf.global_variables_initializer()

sess.run(init)

for step in range(1, num_steps+1):
    
    sess.run(train_op)
    
    if step % step_display == 0 or step == 1:
        # Calculate batch loss and accuracy
        # (note that this consume a new batch of data)
        loss, acc = sess.run([loss_op, accuracy])
        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))

print("Optimization Finished!")

Step 1, Minibatch Loss= 6.1659, Training Accuracy= 0.117
Step 100, Minibatch Loss= 0.3585, Training Accuracy= 0.922
